Ben Trevett 의 [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://github.com/bentrevett/pytorch-seq2seq/blob/master/2%20-%20Learning%20Phrase%20Representations%20using%20RNN%20Encoder-Decoder%20for%20Statistical%20Machine%20Translation.ipynb) 튜토리얼을 한글 데이터셋에 적용해보는 연습이다. 데이터셋은 AI Hub 한국어-영어 번역 말뭉치를 이용한다.

이 모델에서는 [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078) 논문에서 제시한 구조대로 seq2seq 모델을 구현한다.

# 개요

일반적은 인코더-디코더 모델의 구조는 다음과 같다.

![enc-dec](https://github.com/bentrevett/pytorch-seq2seq/raw/d876a1dcacd7aeeeeeaff2c9b806d23116df048f/assets/seq2seq1.png)

직전에 다룬 모델에서는 다음과 같은 다층 LSTM을 이용했다.

![LSTM](https://github.com/bentrevett/pytorch-seq2seq/raw/d876a1dcacd7aeeeeeaff2c9b806d23116df048f/assets/seq2seq4.png)

LSTM 모델의 단점은 디코더가 hidden states에 너무 많은 정보를 저장해야 한다는 것이다. 따라서 이 정보의 부담을 조금 줄여줌으로서 기존 모델을 개선해보고자 한다. 그리고 여기선 LSTM 대신 GRU를 써보자.

# 전처리

저번이랑 비슷함

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator, TabularDataset

import spacy
from konlpy.tag import Mecab
import numpy as np

import random
import math
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

랜덤 시드를 고정하자.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

토크나이저를 지정하자. 여기서 한글은 [KoNLPy](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)의 은전한닢, 영어는 [spaCy](https://spacy.io/)를 이용한다.

In [3]:
mecab = Mecab() # 한글
spacy_en = spacy.load('en') # 영어

토크나이저 함수를 만들자. 이전과는 달리 걍 원래의 순서대로 토큰화한다.

In [4]:
def tokenize_ko(text):
    return [tok for tok in mecab.morphs(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
tokenize_ko('한글은 너무 어렵당!')

['한글', '은', '너무', '어렵', '당', '!']

In [6]:
tokenize_en('Korean is too dificult for me!')

['Korean', 'is', 'too', 'dificult', 'for', 'me', '!']

TorchText의 `Field`를 이용해서 데이터 입력 포맷을 지정하자. `init_token`과 `eos_token` 인자를 이용해서 문장 처음과 마지막에 `<sos>`와 `<eos>` 토큰을 자동으로 추가할 수 있다.

In [8]:
SRC = Field(tokenize = tokenize_ko,
           init_token = '<sos>',
           eos_token = '<eos>',
           )

TRG = Field(tokenize = tokenize_en,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True
           )

In [9]:
fields = {'ko': ('src',SRC), 'en': ('trg',TRG)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [10]:
train_data, test_data = TabularDataset.splits(
                            path = 'data',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)
valid_data = TabularDataset(path = 'data/valid_data.csv',
                            format = 'csv',
                            fields = fields,  )

불러온 데이터는 다음과 같은 형태이다.

In [13]:
vars(train_data[0]), vars(valid_data[0])

({'src': ['선생', '님', '이', '문장', '이', '이해', '가', '안', '가', '요', '.'],
  'trg': ['sir',
   ',',
   'i',
   'do',
   "n't",
   'understand',
   'this',
   'sentence',
   'here',
   '.']},
 {'src': ['학교', '가', '끝나', '자마자', '기숙사', '로', '가요', '.'],
  'trg': ['i',
   'go',
   'to',
   'dormitory',
   'as',
   'soon',
   'as',
   'i',
   'finished',
   'class',
   '.']})

이제 단어장을 만들자. `min_freq` 옵션을 이용하여 최소 2번 이상 등장하는 단어만 사용하도록 하자. 또한 단어장은 검증/테스트셋은 써서는 안된다.

In [14]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

이터레이터를 만들자. 일반적인 `Iterator` 대신 `BucketIterator` 쓰면 입력/출력 배치 안에 있는 문장의 길이가 최대한 비슷하게 되어 패딩을 최소화하게 해준다.

In [15]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.src),
    sort_within_batch = True,
    device = device)

In [16]:
next(iter(train_iterator)).src.shape

torch.Size([12, 64])

# Seq2seq 모델 생성

## 인코더